### Step 1. Download input data

In [ ]:
!rm -rf *
!mkdir example_gtfs_bart
!mkdir outputs
!mkdir outputs/train_outputs
!mkdir outputs/traveler_outputs

!wget "https://raw.githubusercontent.com/cb-cities/transit_sim/main/example_gtfs_bart/stop_times.txt" -O example_gtfs_bart/stop_times.txt
!wget "https://raw.githubusercontent.com/cb-cities/transit_sim/main/example_gtfs_bart/stops.txt" -O example_gtfs_bart/stops.txt
!wget "https://raw.githubusercontent.com/cb-cities/transit_sim/main/example_gtfs_bart/trips.txt" -O example_gtfs_bart/trips.txt

!mkdir model
!wget "https://raw.githubusercontent.com/cb-cities/transit_sim/main/model/transit_sim_model.py" -O model/transit_sim_model.py

!mkdir sp
!wget "https://github.com/UCB-CE170a/Fall2021/raw/master/traffic_data/liblsp.so" -O sp/liblsp.so
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2021/master/traffic_data/interface.py" -O sp/interface.py

!pip install geopandas

### Step 2. Import modules

In [4]:
### plotting
import matplotlib.pyplot as plt

### user module
from model.transit_sim_model import Network, Trains, Travelers

### fix random seed
import numpy as np
np.random.seed(0)

### Step 3. Process GTFS schedules
Need the following tables:
 * stop_times.txt: schedule info
 * trips.txt: map trip_id to route_id
 * stops.txt: get stop coordinates (for visualization)

In [5]:
### read in GTFS files
stop_times_file = 'example_gtfs_bart/stop_times.txt'
trips_file = 'example_gtfs_bart/trips.txt'
stops_file = 'example_gtfs_bart/stops.txt'
### only keep results with this service id
service_id = '2021_09_13-DX-MVS-Weekday-08'

### create all trains from GTFS
all_trains = Trains()
all_nodes, all_links = all_trains.schedule_and_network_from_gtfs(
    stop_times_file, trips_file, stops_file, service_id)

### create network from nodes and links
network = Network(all_nodes, all_links)

In [6]:
### display and export network
display(network.all_nodes.head(3))
display(network.all_links.head(2))
network.all_links.to_csv('outputs/example_links.csv', index=False)
network.all_nodes.to_csv('outputs/example_nodes.csv', index=False)

,route_stop_id,stop_lon,stop_lat,stop_id,type,node_id,geometry
0,5-BERY,-121.874322,37.368759,BERY,platform,0,POINT (-121.87432 37.36876)
1,5-MLPT,-121.890722,37.410562,MLPT,platform,1,POINT (-121.89072 37.41056)
2,5-WARM,-121.939036,37.50257,WARM,platform,2,POINT (-121.93904 37.50257)


,route_stop_id,next_route_stop_id,start_nid,end_nid,initial_weight,geometry
66,5-BERY,5-MLPT,0,1,1.0,"LINESTRING (-121.87432 37.36876, -121.89072 37..."
67,5-MLPT,5-WARM,1,2,1.0,"LINESTRING (-121.89072 37.41056, -121.93904 37..."


### Step 4. Travel demand
 * random demand
 * or, input csv with columns *traveler_id*, *origin_nid*, *destin_nid*, *departure_time*

In [7]:
### random OD
### create instance
travelers = Travelers()
### generate random od
travelers.random_od(network.all_nodes, num_travelers=100)
### find routes
travelers.find_routes(network.network_g, network.station_id_nm_dict)
travelers.set_initial_status(network.station_id_nm_dict)

display(travelers.travelers_df)

,origin_nid,destin_nid,traveler_id,departure_time,traveler_status,update_time,association,next_station,origin_station,destination_station
0,270,261,0,28986,pretrip,0,None,None,all-UCTY,all-PLZA
1,273,226,1,26871,pretrip,0,None,None,all-WDUB,all-12TH
2,226,257,2,29491,pretrip,0,None,None,all-12TH,all-ORIN
3,229,231,3,26830,pretrip,0,None,None,all-24TH,all-ASHB
4,229,256,4,28823,pretrip,0,None,None,all-24TH,all-NCON
...,...,...,...,...,...,...,...,...,...,...
93,249,247,92,28486,pretrip,0,None,None,all-LAFY,all-GLEN
94,241,257,93,27717,pretrip,0,None,None,all-DBRK,all-ORIN
96,247,233,94,28441,pretrip,0,None,None,all-GLEN,all-BAYF
97,274,250,95,26810,pretrip,0,None,None,all-WOAK,all-LAKE


In [8]:
### check if travelers can transfer between lines
#print(travelers.travelers_paths[80])
print(travelers.travelers_key_stops[11])
all_trains.schedule_df[(all_trains.schedule_df['location']=='1-COLM') & 
                      (all_trains.schedule_df['next_time']>29400+140) &
                       (all_trains.schedule_df['next_time']<29424+1000)
                      ]
all_trains.schedule_df[all_trains.schedule_df['trip_id']==964621].head()
display(travelers.travelers_df.iloc[11:12])

{'all-BALB': '2-BALB', '2-BALB': '2-PITT', '2-PITT': 'all-PITT'}


,origin_nid,destin_nid,traveler_id,departure_time,traveler_status,update_time,association,next_station,origin_station,destination_station
11,232,260,11,26731,pretrip,0,None,None,all-BALB,all-PITT


### Step 5. Run the simulation

In [9]:
def save_results(network, trains, travelers, t):
    ### save train results
    train_positions = trains.get_all_train_positions(network)
    train_positions.to_csv('outputs/train_outputs/train_outputs_od100_{}.csv'.format(t), index=False)
    ### save traveler results
    traveler_locations = travelers.travelers_df.groupby(
            ['traveler_status', 'association']).size().to_frame(
            name='num_travelers').reset_index(drop=False)
    traveler_locations.to_csv('outputs/traveler_outputs/traveler_outputs_od100_{}.csv'.format(t), index=False)

In [10]:
for t in range(26664-200, 30700, 20):
    ### update train location
    all_trains.update_location_occupancy(t)
    
    ### update traveler status
    travelers.traveler_update(network, all_trains, t)
    
    ### print and plot results
    #plot_map(network, all_trains, travelers, t)
    save_results(network, all_trains, travelers, t)
    

In [ ]:
### zip outputs and upload to the transit_vis notebook
!tar czvf outputs.tar.gz outputs